In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


In [2]:
os.makedirs("images",exist_ok=True)

In [3]:
n_epochs = 200
batch_size = 64
lr = 0.0002

b1 = 0.5
b2 = 0.999

n_cpu = 8
latent_dim = 100

img_size = 28
channels = 1
sample_interval = 400

In [4]:
img_shape = (channels, img_size, img_size)

In [5]:
cuda = True if torch.cuda.is_available() else False

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [8]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [9]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [10]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [11]:
# ----------
#  Training
# ----------

for epoch in range(n_epochs):
    for i, (imgs,a) in enumerate(dataloader):
        #print(a)

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        #print(real_imgs.size())

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval/2 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

        
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images-fashion/%d.png" % batches_done, nrow=5, normalize=True)

[Epoch 0/200] [Batch 0/938] [D loss: 0.694152] [G loss: 0.708097]
[Epoch 0/200] [Batch 400/938] [D loss: 0.351821] [G loss: 1.369241]
[Epoch 0/200] [Batch 800/938] [D loss: 0.213687] [G loss: 1.907099]
[Epoch 1/200] [Batch 262/938] [D loss: 0.279028] [G loss: 1.460900]
[Epoch 1/200] [Batch 662/938] [D loss: 0.322412] [G loss: 1.495587]
[Epoch 2/200] [Batch 124/938] [D loss: 0.353952] [G loss: 1.227704]
[Epoch 2/200] [Batch 524/938] [D loss: 0.323709] [G loss: 2.621522]
[Epoch 2/200] [Batch 924/938] [D loss: 0.386572] [G loss: 2.849360]
[Epoch 3/200] [Batch 386/938] [D loss: 0.353470] [G loss: 1.331467]
[Epoch 3/200] [Batch 786/938] [D loss: 0.379417] [G loss: 1.114637]
[Epoch 4/200] [Batch 248/938] [D loss: 0.247174] [G loss: 1.577502]
[Epoch 4/200] [Batch 648/938] [D loss: 0.403204] [G loss: 1.846150]
[Epoch 5/200] [Batch 110/938] [D loss: 0.318189] [G loss: 1.662735]
[Epoch 5/200] [Batch 510/938] [D loss: 0.434388] [G loss: 0.899238]
[Epoch 5/200] [Batch 910/938] [D loss: 0.338910] [